In [ ]:

!pip uninstall -y sentence-transformers
!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q "transformers==4.43.2" "datasets==2.18.0" "accelerate==0.29.3" "peft==0.10.0" "bitsandbytes==0.43.1" "trl==0.8.6" "protobuf==3.20.3"
!pip install -q einops scipy sentencepiece tensorboard

import os
os.kill(os.getpid(), 9)

Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 126.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 121.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import os

try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Successfully logged into Hugging Face.")
except Exception as e:
    print(f"Login failed. Please ensure 'HF_TOKEN' is set in Colab Secrets and has WRITE permissions. Error: {e}")
    raise SystemExit("Stopping due to failed Hugging Face login.")

✅ Successfully logged into Hugging Face.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os

base_model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
adapter_path_on_drive = "/content/drive/MyDrive/Hernia-Model/llama3-hernia-analyst-A100-v1"
new_model_repo_on_hub = "Laxmikant17/Llama-3-8B-Hernia-Analyst-600-Patients-8k"

print(f"Verifying adapter path: {adapter_path_on_drive}")
if not os.path.isdir(adapter_path_on_drive):
    print(f"FATAL ERROR: The directory does not exist at '{adapter_path_on_drive}'.")
    raise SystemExit("Stopping.")
if not os.path.exists(os.path.join(adapter_path_on_drive, "adapter_config.json")):
     print(f"FATAL ERROR: Cannot find 'adapter_config.json' in the directory.")
     raise SystemExit("Stopping.")
print("Adapter path and files verified successfully.")


print(f"\nLoading base model: {base_model_name}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)


print(f"Loading LoRA adapter from Google Drive...")


offload_folder = "./temp_offload"
os.makedirs(offload_folder, exist_ok=True)

try:
    model = PeftModel.from_pretrained(base_model, adapter_path_on_drive, offload_folder=offload_folder)
    print("Merging the adapter with the base model... This is a memory-intensive step.")
    model = model.merge_and_unload()
    print("Model merged successfully.")
except Exception as e:
    print(f"ERROR: Failed to load the adapter. Error details: {e}")
    raise SystemExit("Stopping.")

print(f"\n🚀 Uploading the final model to the Hugging Face Hub at: {new_model_repo_on_hub}")
print("This will take some time, as the model is large...")
try:
    model.push_to_hub(new_model_repo_on_hub, create_repo=True, private=False)
    tokenizer.push_to_hub(new_model_repo_on_hub, create_repo=True, private=False)

    print(f"\n\n HUGE CONGRATULATIONS! Your 600-patient model is now live on the Hub: https://huggingface.co/{new_model_repo_on_hub}")
except Exception as e:
    print(f"\n An error occurred during the upload process. Error details: {e}")


# Clean up the temporary folder to save disk space
import shutil
shutil.rmtree(offload_folder)
print(f"\n🧹 Cleaned up temporary offload folder.")

Verifying adapter path: /content/drive/MyDrive/Hernia-Model/llama3-hernia-analyst-A100-v1
✅ Adapter path and files verified successfully.

Loading base model: meta-llama/Meta-Llama-3-8B-Instruct


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]



✅✅✅ HUGE CONGRATULATIONS! Your 600-patient model is now live on the Hub: https://huggingface.co/Laxmikant17/Llama-3-8B-Hernia-Analyst-600-Patients-8k

🧹 Cleaned up temporary offload folder.
